In [30]:
import pandas as pd
from bs4 import BeautifulSoup
import re
from IPython.display import HTML

# Read the file
file = pd.read_json('../raw_data.json')
series = pd.Series(file['data'])

# convert json to dataframe
df = pd.json_normalize(series)

# print dataframe
# df

In [39]:
counter = 0

# salaries_df = pd.DataFrame({'name': [], 'netto': [], 'brutto': [], 'date': [], 'currency': [], 'period': []})
salaries_df = pd.DataFrame({'name': [], 'salary': [], 'date': [], 'currency': [], 'period': [], 'url': []})
zarobki_lines = []

salary_keywords = ['Zarobki', 'zarobki', 'ZAROBKI']
currency_keywords = ['ZŁ', 'Zł', 'zł', 'USD','Usd', 'usd', '$','EURO', 'Euro', 'euro', 'EUR', 'Eur', 'eur', '€']
thousands = ['k']

# iterate over all posts
# analyze every post
for index, post in df.iterrows():
    soup = BeautifulSoup(post['post_content'], 'html.parser')
    
    # remove blockquotes and avoid repetition
    for blockquote in soup.find_all('blockquote'):
        blockquote.decompose()
        
    # get clean text without html tags
    clean_text = soup.get_text()
    
    # search through the every line in post
    for line in clean_text.split('\n'):
        for keyword in salary_keywords:
            
            # check if salary is present
            if keyword in line:
                new_row = {'name': post['author'], 'salary': 0, 'url': post['url']}
                
                # regexp rules
                full_salary_match = re.search(r'\d+', line)
                short_salary_match = re.search(r'\d+([.,]?)\s*k', line)
                
                if(full_salary_match):
                    new_row['salary'] = full_salary_match.group(0)
                    salaries_df.loc[len(salaries_df)] = new_row
                    
                if (short_salary_match):
                    new_row['salary'] = short_salary_match.group(0)
                    salaries_df.loc[len(salaries_df)] = new_row
                
                
print(len(salaries_df))
HTML(salaries_df.to_html(render_links=True, escape=False))

3695


,name,salary,date,currency,period,url
0,zarobk,6000,NaN,NaN,NaN,https://4programmers.net/Forum/Kariera/233131-ile_zarabiacie?p=1031091#id1031091
1,sssss,5500,NaN,NaN,NaN,https://4programmers.net/Forum/Kariera/233131-ile_zarabiacie?p=1031101#id1031101
2,franck,5000,NaN,NaN,NaN,https://4programmers.net/Forum/Kariera/233131-ile_zarabiacie?p=1031459#id1031459
3,Hejmdndn,8000,NaN,NaN,NaN,https://4programmers.net/Forum/Kariera/233131-ile_zarabiacie?p=1031465#id1031465
4,piotrp,350,NaN,NaN,NaN,https://4programmers.net/Forum/Kariera/233131-ile_zarabiacie?p=1031900#id1031900
5,KtośZKrakowa,3300,NaN,NaN,NaN,https://4programmers.net/Forum/Kariera/233131-ile_zarabiacie?p=1032027#id1032027
6,_noname_,5700,NaN,NaN,NaN,https://4programmers.net/Forum/Kariera/233131-ile_zarabiacie?p=1032036#id1032036
7,ne0,9000,NaN,NaN,NaN,https://4programmers.net/Forum/Kariera/233131-ile_zarabiacie?p=1035672#id1035672
8,kdkrk,12000,NaN,NaN,NaN,https://4programmers.net/Forum/Kariera/233131-ile_zarabiacie?p=1035685#id1035685
9,post_inkognito,5900,NaN,NaN,NaN,https://4programmers.net/Forum/Kariera/233131-ile_zarabiacie?p=1031491#id1031491
